# Этап 1. Парсинг данных

В этом разделе собираем данные для дальнейшего анализа.

[Источник](https://www.imdb.com/search/title/?title_type=feature)

In [ ]:
!pip install requests
!pip install beautifulsoup4`

import requests
from bs4 import BeautifulSoup
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


Создадим функцию, которая будет собирать необходиимые данные с 1 страницы (на странице 50 фильмов).

In [11]:
def parse_imdb_page():
    while True:
        url = yield
        response = requests.get(url=url, headers={'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'})
        if not response.ok:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        all_films = soup.find_all('div', class_='lister-item mode-advanced')
        # 'div' - тег блока, под которым у нас вся инфа о фильме. class_ - класс блока

        parsed_films_list = []

        for film in all_films:
            if (film_content := film.find('div', class_='lister-item-content')) is not None:
                film_header = film_content.find('h3', class_='lister-item-header')
                film_name = film_header.find('a').text.strip()
                film_url = film_header.find('a')['href']
                film_global_rate = film_header.find('span', class_='lister-item-index unbold text-primary').text.strip()
                film_year = film_header.find('span', class_='lister-item-year text-muted unbold').text.strip()
                film_rates = film_content.find('div', class_='ratings-bar')

                try: # Не у всех фильмов есть отметки на meta score & imdb. Поэтому пробуем их найти
                    film_imdb_rate = film_rates.find('div', class_='inline-block ratings-imdb-rating')['data-value'] # rate по imdb
                    film_meta_score_rate = film_rates.find('div', class_='inline-block ratings-metascore').find('span').text.strip() # rate на meta score
                except:
                    film_imdb_rate = None
                    film_meta_score_rate = None

                film_muted_spans = film_content.find_all('p', class_='text-muted')

                for film_desc in film_muted_spans:
                    genre = film_desc.find('span', class_='genre')
                    duration = film_desc.find('span', class_='runtime')
                    if genre or duration:
                        film_duration = None if not duration else duration.text.strip()
                        if genre is not None:
                            film_genre = genre.text.strip()
                        else:
                            film_genre = None
                    else:
                        film_description = film_desc.text.strip()

                try: # Не у всех фильмов есть кол-во голосов, так что пробуем их найти
                    film_votes = film_content.find('p', class_='sort-num_votes-visible').find(
                        'span',
                        attrs={'name': 'nv'}
                    )['data-value']
                except:
                    film_votes = None

                parsed_films_list.append({
                    'name': film_name,
                    'url': film_url,
                    'global_rate': film_global_rate,
                    'release_year': film_year,
                    'imdb_rate': film_imdb_rate,
                    'metascore_rate': film_meta_score_rate,
                    'description': film_description,
                    'votes': film_votes,
                    'duration': film_duration,
                    'genre': film_genre
                })

        yield parsed_films_list


Соберем функцию, чтобы мы могли спокойно переключаться между страницами и собрать большую базу.

In [12]:
def get_result(max_count: int = 5000):
    cour = parse_imdb_page()
    next(cour)
    info = cour.send('https://www.imdb.com/search/title/?title_type=feature')
    for count in range(51, max_count, 50):
        next_url = f'https://www.imdb.com/search/title/?title_type=feature&start={count}&ref_=adv_nxt'
        next(cour)
        info += cour.send(next_url)
    else:
        cour.close()
    return info
result = get_result()


Соберем наши данные в файл:

In [13]:

df = pd.DataFrame(result, columns=list(result[0].keys()))
df.to_csv('list.csv', index=False)

Данные лежат в файле list.csv, уберем в табличке ссылки на фильмы и будет готово!

In [14]:
df = df.drop('url', axis=1)
df.head()

,name,global_rate,release_year,imdb_rate,metascore_rate,description,votes,duration,genre
0,Человек-паук: Паутина вселенных,1.,(2023),9,86,"Miles Morales catapults across the Multiverse,...",112055,140 min,"Animation, Action, Adventure"
1,Русалочка,2.,(I) (2023),"7,2",59,A young mermaid makes a deal with a sea witch ...,79594,135 min,"Adventure, Family, Fantasy"
2,Трансформеры: Восхождение Звероботов,3.,(2023),"6,6",42,"During the '90s, a new faction of Transformers...",16401,127 min,"Action, Adventure, Sci-Fi"
3,Неудержимые 4,4.,(2023),None,None,The Expendables will square up against: an arm...,None,None,"Action, Adventure, Thriller"
4,Человек-паук: Через Вселенные,5.,(2018),"8,4",87,Teen Miles Morales becomes the Spider-Man of h...,584309,117 min,"Animation, Action, Adventure"
